<a href="https://colab.research.google.com/github/arunchow/ClassTest1/blob/master/Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
"""
Student Information System (SIS)
--------------------------------
A menu-driven CLI application to register, view, and search students with
persistent storage using a CSV file.

Features:
- Load data on startup; save on exit (or exit without saving).
- Register students (name, age, 3 subject grades).
- Compute average and classify as "Excellent", "Good", or "Needs Improvement".
- View all students.
- Search by name (case-insensitive).

Structure:
- Clear separation of concerns across file I/O, user interaction, and data logic.
- Uses Python's csv.DictReader/DictWriter for persistence.
- Input validation and helpful error messages.

Author: Arun Chowdhury
Date: 9th, November 2025
"""

from __future__ import annotations

import csv
from pathlib import Path
from typing import Dict, List, Tuple

# =========================
# ----- Configuration -----
# =========================

CSV_FILENAME = "students.csv"
CSV_HEADERS = ["name", "age", "grade1", "grade2", "grade3", "average", "classification"]
# Global Min Age
MIN_AGE = 16
# Global Max Age
MAX_AGE = 100
# Global Min Grade
MIN_GRADE = 0.0
# Global Max Grade
MAX_GRADE = 100.0
# Classification thresholds (inclusive for lower bound)
EXCELLENT_MIN = 90.0
GOOD_MIN = 75.0
# Classification Types
EXCELLENT = "Excellent"
GOOD = "Good"
NEEDS_IMPROVEMENT = "Needs Improvement"
# Otherwise -> Needs Improvement


# ===========================================
# ----- Computation and Classification  -----
# ===========================================

def compute_average_and_classification(grades: Tuple[float, float, float]) -> Tuple[float, str]:
    """
    Compute the average grade and a classification string.

    Classification rules:
        - >= EXCELLENT_MIN: "Excellent"
        - >= GOOD_MIN: "Good"
        - else          : "Needs Improvement"

    Args:
        grades: A tuple of three numeric grades.

    Returns:
        (average, classification)
    """
    g1, g2, g3 = grades
    avg = (g1 + g2 + g3) / 3.0
    if avg >= EXCELLENT_MIN:
        cls = EXCELLENT
    elif avg >= GOOD_MIN:
        cls = GOOD
    else:
        cls = NEEDS_IMPROVEMENT
    return round(avg, 2), cls


def sanitize_csv_row(row: Dict[str, str]) -> Dict[str, str]:
    """
    Convert and sanitize CSV values into expected types/strings
    while keeping them CSV-writable.

    Args:
        row: Raw row from csv.DictReader.

    Returns:
        A cleaned dict with expected keys and string values.
    """
    cleaned: Dict[str, str] = {}
    cleaned["name"] = (row.get("name") or "").strip()

    # Convert numeric fields carefully; default to 0 if conversion fails.
    def to_int(value: str, default: int = 0) -> int:
        try:
            return int(float(value))
        except (TypeError, ValueError):
            return default

    def to_float(value: str, default: float = 0.0) -> float:
        try:
            return float(value)
        except (TypeError, ValueError):
            return default

    age = to_int(row.get("age", "0"), 0)
    g1 = to_float(row.get("grade1", "0"))
    g2 = to_float(row.get("grade2", "0"))
    g3 = to_float(row.get("grade3", "0"))

    # If average/classification missing or wrong, recompute
    avg, cls = compute_average_and_classification((g1, g2, g3))

    cleaned["age"] = str(age)
    cleaned["grade1"] = f"{g1:.2f}"
    cleaned["grade2"] = f"{g2:.2f}"
    cleaned["grade3"] = f"{g3:.2f}"
    cleaned["average"] = f"{avg:.2f}"
    cleaned["classification"] = cls
    return cleaned


# =======================================================
# ----- File Handling CSV_FILENAME defined in config-----
# =======================================================

def load_students_from_file(filename: str = CSV_FILENAME) -> List[Dict[str, str]]:
    """
    Load student records from a CSV file into a list of dictionaries.

    Args:
        filename: Path to the CSV file.

    Returns:
        A list of student dictionaries. If the file does not exist,
        returns an empty list and prints an informational message.
    """
    students: List[Dict[str, str]] = []
    path = Path(filename)
    if not path.exists():
        print(f"[i] No existing data found at '{filename}'. Starting with an empty dataset.")
        return students

    with path.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        # Accept any CSV with at least name and numeric fields; sanitize rows
        for row in reader:
            students.append(sanitize_csv_row(row))

    print(f"[✓] Loaded {len(students)} student(s) from '{filename}'.")
    return students

#def save_students_to_file(students: List[Dict[str, str]], filename: str = CSV_FILENAME) -> None:
def save_students_to_file(students: List[Dict[str, str]], filename: str = CSV_FILENAME):
    """
    Save student records to a CSV file using csv.DictWriter.

    Args:
        students: List of student dictionaries (strings for all values).
        filename: Destination CSV file name.

    Returns:
        None. Writes the file to disk.
    """
    path = Path(filename)
    path.parent.mkdir(parents=True, exist_ok=True)

    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_HEADERS)
        writer.writeheader()
        for s in students:
            # Ensure consistent keys & types
            row = sanitize_csv_row(s)
            writer.writerow(row)

    print(f"Student data saved successfully.\n")


# ====================================================
# ----- Functions to check for non empty strings -----
# ====================================================

def check_nonempty_string(inp: str) -> str:
    """
    Prompt for a non-empty string.

    Args:
        prompt: The prompt to display.

    Returns:
        A non-empty trimmed string.
    """
    while True:
        inp_str = input(inp).strip()
        if inp_str:
            return inp_str
        print("  [!] Input string cannot be empty. Please try again.")


def check_int(inp: str, min_value: int | None = None, max_value: int | None = None) -> int:
    """
    Prompt for an integer with optional bounds.

    Args:
        inp: The prompt text.
        min_value: Optional minimum (inclusive). Defined in config.
        max_value: Optional maximum (inclusive). Defined in config.

    Returns:
        The validated integer value.
    """
    while True:
        inp_str = input(inp).strip()
        try:
            value = int(inp_str)
            if min_value is not None and value < min_value:
                print(f"  [!] Value must be >= {min_value}.")
                continue
            if max_value is not None and value > max_value:
                print(f"  [!] Value must be <= {max_value}.")
                continue
            return value
        except ValueError:
            print("  [!] Please enter a valid integer.")


def check_float(inp: str, min_value: float | None = None, max_value: float | None = None) -> float:
    """
    Prompt for a float with optional bounds.

    Args:
        prompt: The prompt text.
        min_value: Optional minimum (inclusive). Defined in config.
        max_value: Optional maximum (inclusive). Defined in config.
    Returns:
        The validated float value.
    """
    while True:
        inp_str = input(inp).strip()
        try:
            value = float(inp_str)
            if min_value is not None and value < min_value:
                print(f"  [!] Value must be >= {min_value}.")
                continue
            if max_value is not None and value > max_value:
                print(f"  [!] Value must be <= {max_value}.")
                continue
            return value
        except ValueError:
            print("  [!] Please enter a valid number.")


def register_student(students: List[Dict[str, str]]) -> None:
    """
    Register a new student and append to the list.

    Prompts the user for:
        - name (str, non-empty)
        - age (int, 3..120)
        - three grades (floats, 0..100)

    Computes average and classification, then prints the result.
    Marks data as changed by mutating the given list.
    Passes Minimum and Maximum allowed age values to check_int
    Passes Minimum and Maximum allowed grades values to check_float.

    Args:
        students: The list of student records (dicts of strings).
    """
    print("\n== Register a New Student ==")
    name = check_nonempty_string(" Enter student name: ")
    age = check_int(" Enter student age: ", MIN_AGE, MAX_AGE)
    g1 = check_float(" Enter grade for subject 1: ", MIN_GRADE, MAX_GRADE)
    g2 = check_float(" Enter grade for subject 2: ", MIN_GRADE, MAX_GRADE)
    g3 = check_float(" Enter grade for subject 3:: ", MIN_GRADE, MAX_GRADE)

    avg, cls = compute_average_and_classification((g1, g2, g3))

    record = {
        "name": name,
        "age": str(age),
        "grade1": f"{g1:.2f}",
        "grade2": f"{g2:.2f}",
        "grade3": f"{g3:.2f}",
        "average": f"{avg:.2f}",
        "classification": cls,
    }
    students.append(record)

    print(f"\n{name} has been registered with an average grade of {avg:.2f} - {cls}\n")

#Needs work/corrections
def view_students(students: List[Dict[str, str]]) -> None:
    """
    This function displays all students stored in the students list,
    showing their name, age, average grade, and grades for each subject
    Args:
        students: The list of student records (dicts of strings).

    Returns:
        None. Prints to stdout.
    """
    print("\n== All Students ==")
    if not students:
        print("  (No students found in the csv file.)\n")
        return

    # Column widths defined for each attribute
    #headers = ["Name", "Age", "Grade1", "Grade2", "Grade3", "Average", "Class"]
    #widths = [20, 5, 8, 8, 8, 8, 18]
    #line = " ".join(h.ljust(w) for h, w in zip(headers, widths))
    #print(line)
    #print("-" * len(line))

    #for std in students:
    #    row = [
    #        (std.get("name", "") or "")[:widths[0]].ljust(widths[0]),
    #        (std.get("age", "") or "").rjust(widths[1]),
    #        (std.get("grade1", "") or "").rjust(widths[2]),
    #        (std.get("grade2", "") or "").rjust(widths[3]),
    #        (std.get("grade3", "") or "").rjust(widths[4]),
    #        (std.get("average", "") or "").rjust(widths[5]),
    #        (std.get("classification", "") or "")[:widths[6]].ljust(widths[6]),
    #    ]
    for std in students:
        print(f"Name: {std.get("name", "")}, Age: {std.get("age", "")}, Average Grade: {std.get("average", "")}\n")
        print(f"Grades: {std.get("grade1", "")}, {std.get("grade2", "")}, {std.get("grade3", "")}\n")
        print(f"------\n")

    print()


def search_student(students: List[Dict[str, str]]) -> None:
    """
    This function prompts the user to search for a student
    by name and displays the student’s information if
    they are found in the csv file and the list that was passed

    Search is case-insensitive and returns the first exact name match.

    Args:
        students: The list of student records.

    Returns:
        None. Prints to stdout.
    """
    print("\n== Search Student ==")
    query = check_nonempty_string("  Enter name to search: ").lower()

    for std in students:
        if std.get("name", "").lower() == query:
            print("\n[We found your record in the csv file!]")
            print(f"  Name:  {std.get('name', '')}")
            print(f"  Age:   {std.get('age', '')}")
            print(f"  Grade for subject 1:{std.get('grade1', '')}")
            print(f"  Grade for subject 2:{std.get('grade2', '')}")
            print(f"  Grade for subject 3:{std.get('grade3', '')}")
            print(f"  Avg:   {std.get('average', '')}")
            print(f"  Class: {std.get('classification', '')}\n")
            return

    print("  [!] No student found with the name provided in the csv file.\n")


def display_menu() -> None:
    """
    This function displays the CLI Menu options to the user,
    allowing them to choose what action to perform
    (register, view, search, save and exit, exit without saving).

    Returns:
        None.
    """
    print("======== Student Information System ========")
    print("  1) Register a New Student")
    print("  2) View All Students")
    print("  3) Search for a Student by Name")
    print("  4) Save and Exit")
    print("  5) Exit without Saving")
    print("============================================")


# =========================
# --------- Main ----------
# =========================

def main() -> None:
    """
    This is the main loop that initializes the student data,
    shows the menu to the user, and processes their input.
	  Depending on the user's choice, the program calls the
    appropriate function.

    """
    students = load_students_from_file(CSV_FILENAME)
    print(f"Student data loaded successfully.")
    dirty = False

    while True:
        display_menu()
        choice = input("Select an option (1-5): ").strip()

        if choice == "1":
            register_student(students)
            dirty = True

        elif choice == "2":
            view_students(students)

        elif choice == "3":
            search_student(students)

        elif choice == "4":
            # Save and Exit
            if dirty:
                save_students_to_file(students, CSV_FILENAME)
                print(f"Exiting program. Data saved.\n")
            else:
                print("Exiting program. No changes to save.\n")
            #print("[✓] Changes saved. Goodbye!")

            break

        elif choice == "5":
            # Exit without saving (confirm if dirty)
            if dirty:
                confirm = input("You have unsaved changes. Exit without saving? (Y/N): ").strip().lower()
                if confirm != "Y":
                    print("  [i] Cancelled exit. Returning to menu.\n")
                    continue
            print("Exiting the Student CLI Application!")
            break

        else:
            print("  [!] Invalid selection. Please choose 1, 2, 3, 4, or 5.\n")


if __name__ == "__main__":
    main()


[✓] Loaded 5 student(s) from 'students.csv'.
Student data loaded successfully.
======== Student Information System ========
  1) Register a New Student
  2) View All Students
  3) Search for a Student by Name
  4) Save and Exit
  5) Exit without Saving
Select an option (1-5): 2

== All Students ==
Name                 Age   Grade1   Grade2   Grade3   Average  Class             
---------------------------------------------------------------------------------
Name: John, Age: 21, Average Grade: 98.00

Grades: 99.00, 98.00, 97.00

------

Name: Derek, Age: 19, Average Grade: 60.33

Grades: 0.00, 90.00, 91.00

------

Name: Monica, Age: 21, Average Grade: 87.00

Grades: 90.00, 86.00, 85.00

------

Name: Rita, Age: 20, Average Grade: 83.67

Grades: 78.00, 89.00, 84.00

------

Name: Kirk, Age: 19, Average Grade: 92.33

Grades: 98.00, 99.00, 80.00

------


======== Student Information System ========
  1) Register a New Student
  2) View All Students
  3) Search for a Student by Name
  4)